The `pyrit.memory` module provides functionality to keep track of the conversation history. In a nutshell, this can be used as follows

The PyRIT DuckDB database comprises of tables defined in the `pyrit.memory.memory_models` module.



In [1]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT license.

from uuid import uuid4
from pyrit.memory import DuckDBMemory
from pyrit.models import PromptRequestPiece

conversation_id = str(uuid4())

memory = DuckDBMemory()
message_list = [
    PromptRequestPiece(
        role="user", original_prompt_text="Hi, chat bot! This is my initial prompt.", conversation_id=conversation_id
    ),
    PromptRequestPiece(
        role="assistant", original_prompt_text="Nice to meet you! This is my response.", conversation_id=conversation_id
    ),
    PromptRequestPiece(
        role="user",
        original_prompt_text="Wonderful! This is my second prompt to the chat bot!",
        conversation_id=conversation_id,
    ),
]

memory.add_request_pieces_to_memory(request_pieces=message_list)


entries = memory.get_chat_messages_with_conversation_id(conversation_id=conversation_id)

for entry in entries:
    print(entry)

role='user' content='Hi, chat bot! This is my initial prompt.' name=None tool_calls=None tool_call_id=None
role='assistant' content='Nice to meet you! This is my response.' name=None tool_calls=None tool_call_id=None
role='user' content='Wonderful! This is my second prompt to the chat bot!' name=None tool_calls=None tool_call_id=None


In [2]:

# update based on conversation_id
update_fileds = {"converted_prompt_text": "this is updated field"}
memory.update_entries_by_conversation_id(conversation_id=conversation_id, update_fields=update_fileds)


prompt_entries = memory.get_prompt_entries_with_conversation_id(conversation_id=conversation_id)

for entry in prompt_entries:
    print(entry)

user: this is updated field
assistant: this is updated field
user: this is updated field


In [3]:
# Cleanup memory resources
memory.dispose_engine()